In [1085]:
%%writefile vectors.hpp
#pragma once
#include <iostream>
#include <Eigen/Dense>

using Eigen6Vector = Eigen::Matrix<float, 6, 1>;
using Eigen6by6Matrix = Eigen::Matrix<float, 6, 6>;
using Eigen3by3Matrix = Eigen::Matrix<float, 3, 3>;

class Vector3 {

  public:
    __device__ Vector3();
    Vector3(float, float, float);
    float x;
    float y;
    float z;
    __device__ void normalize();
};

class Vector6 {

  public:
    __device__ Vector6(void);
    __device__ Vector6(float, float, float, float, float, float);
    __device__ Eigen6Vector to_eigen(void);
    float x, y, z, p, q, w;
};

Overwriting vectors.hpp


In [1086]:
%%writefile vectors.cu
#include "vectors.hpp"

__device__ void Vector3::normalize() {

    float magnitude = (x * x) + (y * y) + (z * z);
    x = x / magnitude;
    y = y / magnitude;
    z = z / magnitude;
}

__device__ Vector3::Vector3(): x(0.), y(0.), z(0.) {
}

Vector3::Vector3(float x, float y, float z): x(x), y(y), z(z) {
}

__device__ Vector6::Vector6(void): x(0.), y(0.), z(0.), p(0.), q(0.), w(0.) {

}

__device__ Vector6::Vector6(float x, float y, float z, float p, float q, float w): x(x), y(y), z(z), p(p), q(q), w(w) {

}

__device__ Eigen6Vector Vector6::to_eigen(void) {

  Eigen6Vector vector;
  vector << x, y, z, p, q, w;
  return vector;
}

Overwriting vectors.cu


In [1087]:
%%writefile screw.hpp
#pragma once
#include "vectors.hpp"

class Screw {

  public:
    Screw();
    Screw(Vector3, Vector3);
    //Screw(float*);

    Vector3 w;
    Vector3 v;
};

Overwriting screw.hpp


In [1088]:
%%writefile screw.cu
#include "screw.hpp"
Screw::Screw(): w(Vector3(0., 0., 0.)), v(Vector3(0., 0., 0.)) {
}

Screw::Screw(Vector3 w, Vector3 v): w(w), v(v) {
}

Overwriting screw.cu


In [1089]:
%%writefile transformation.hpp
#pragma once
#include "screw.hpp"
#include "vectors.hpp"
#include <iostream>
#include <Eigen/Dense>


//using Eigen6Vector = Eigen::Matrix<float, 6, 1>;
//using Eigen6by6Matrix = Eigen::Matrix<float, 6, 6>;
//using Eigen3by3Matrix = Eigen::Matrix<float, 3, 3>;


inline __device__ float dot_product(float* row, float* col, int size) {

  float result = 0.;
  for (int i = 0; i < size; i++) {
    result += (row[i] * col[i]);
  }

  return result;
}

class Transformation {

  public:
    static const int num_entries_in_matrix = 16;
    float values[num_entries_in_matrix];

    __device__ Transformation();
    __device__ ~Transformation();
    __device__ Transformation(float*);
    __device__ Transformation operator+(const Transformation&);
    __device__ Transformation operator*(const Transformation&);
    __device__ Transformation operator*(const float);
    __device__ Transformation bracket_w(const Vector3& w);
    __device__ Transformation bracket_w_squared(const Vector3& w);
    __device__ Vector3 operator*(const Vector3&);
    __device__ void set_translation(const Vector3&);
    __device__ float* get_row(int) const;
    __device__ float* get_column(int) const;
    __device__ Vector3 euler_angles(void) const;
    __device__ Transformation& operator=(const Transformation&);
    __device__ Transformation(Transformation&);
    __device__ Vector6 pose(void) const;
    __device__ Vector3 translatation(void) const;
    //void print() const;
};

Overwriting transformation.hpp


In [1090]:
%%writefile transformation.cu
#include "transformation.hpp"

__device__ Vector3 Transformation::translatation(void) const {

  Vector3 result;

  result.x = values[3];
  result.y = values[7];
  result.z = values[11];

  return result;
}

__device__ Vector6 Transformation::pose(void) const {
  float x, y, z;

  x = values[3];
  y = values[7];
  z = values[11];

  Vector3 angles = euler_angles();

  return Vector6(x, y, z, angles.x, angles.y, angles.z);
  //return Vector6(x, y, z, x, y, z);
}

__device__ Transformation::Transformation(Transformation& right) {

  for (int i = 0; i < num_entries_in_matrix; i++) {
      values[i] = right.values[i];
  }
}

 __device__ Transformation& Transformation::operator=(const Transformation& right) {

    for (int i = 0; i < num_entries_in_matrix; i++) {
      values[i] = right.values[i];
    }

    return *this;
  }

__device__ Vector3 Transformation::euler_angles(void) const {
  // https://lavalle.pl/planning/node103.html

  Vector3 result;

  float r11 = values[0];
  float r21 = values[4];
  float r31 = values[8];
  float r32 = values[9];
  float r33 = values[10];

  result.x = atan2f(r21, r11); // atan2f(r11, r21);
  result.y = atan2f(-r31, sqrtf((r32 * r32) + (r33 * r33))); // atan2f(sqrtf((r32 * r32) + (r33 * r33)), -r31);
  result.z = atan2f(r32, r33); // atan2f(r33, r32);

  return result;
}

__device__ Transformation::Transformation() {

  // Create the identity matrix.
  for (int i = 0; i < num_entries_in_matrix; i++) {
    values[i] = 0.0;
  }

  values[0] = 1.;
  values[5] = 1.;
  values[10] = 1.;
  values[15] = 1.;
}

__device__ Transformation::~Transformation() {
  //free(values);
}

__device__ Transformation::Transformation(float* input_values) {

  for (int i = 0; i < num_entries_in_matrix; i++) {
    this->values[i] = input_values[i];
  }
}

__device__ Transformation Transformation::operator+(const Transformation &right) {

  Transformation result = Transformation();

  for (int i = 0; i < num_entries_in_matrix; i++) {
    result.values[i] = values[i] + right.values[i];
  }

  result.values[num_entries_in_matrix - 1] = 1.0;
  return result;
}

__device__ Transformation Transformation::operator*(const Transformation &right) {

  Transformation result = Transformation();
  int num_entries_in_row = 4;

  for (int row_index = 0; row_index < num_entries_in_row; row_index++) {
    float* row = get_row(row_index);

    for (int col_index = 0; col_index < num_entries_in_row; col_index++) {
      float* column = right.get_column(col_index);
      result.values[(row_index * 4) + col_index] = dot_product(row, column, num_entries_in_row);
      free(column);
    }
    free(row);
  }

  return result;
}

__device__ Transformation Transformation::operator*(const float c) {

  Transformation result = Transformation();

  for (int i = 0; i < num_entries_in_matrix - 1; i++) {
    result.values[i] = values[i] * c;
  }

  return result;
}

__device__ Transformation Transformation::bracket_w(const Vector3& w) {

  Transformation result = Transformation();

  // First row.
  result.values[0] = 0.;
  result.values[1] = -w.z;
  result.values[2] = w.y;

  // Second row.
  result.values[4] = w.z;
  result.values[5] = 0;
  result.values[6] = -w.x;

  // Third row.
  result.values[8] = -w.y;
  result.values[9] = w.x;
  result.values[10] = 0.;

  return result;
}

__device__ Transformation Transformation::bracket_w_squared(const Vector3& w) {

  Transformation result = Transformation();

  // First row.
  result.values[0] = -(w.z * w.z) - (w.y * w.y);
  result.values[1] = w.y * w.x;
  result.values[2] = w.z * w.x;

  // Second row.
  result.values[4] = w.y * w.x;
  result.values[5] = -(w.z * w.z) - (w.x * w.x);
  result.values[6] = w.z * w.y;

  // Third row.
  result.values[8] = w.z * w.x;
  result.values[9] = w.z * w.y;
  result.values[10] = -(w.y * w.y) - (w.x * w.x);

  return result;
}

__device__ Vector3 Transformation::operator*(const Vector3& right) {

  Vector3 result = Vector3();
  result.x = (values[0] * right.x) + (values[1] * right.y) + (values[2] * right.z);
  result.y = (values[4] * right.x) + (values[5] * right.y) + (values[6] * right.z);
  result.z = (values[8] * right.x) + (values[9] * right.y) + (values[10] * right.z);
  return result;
}

__device__ void Transformation::set_translation(const Vector3& v) {
  values[3] = v.x;
  values[7] = v.y;
  values[11] = v.z;
}

__device__ float* Transformation::get_row(int index) const {

  float* row = ((float*)values) + (index * 4);
  float* copy = (float*) malloc(sizeof(float) * 4);
  for (int i = 0; i < 4; i++) {
    copy[i] = row[i];
  }

  return copy;
}

__device__ float* Transformation::get_column(int index) const {

  float* column = (float*) malloc(sizeof(float) * 4);
  for (int i = 0; i < 4; i++) {
    column[i] = values[index + (4 * i)];
  }

  return column;
}

//void Transformation::print() const;

Overwriting transformation.cu


In [1091]:
%%writefile main.cu
#include "transformation.hpp"
#include "vectors.hpp"
#include "screw.hpp"

#include <iostream>
#include <Eigen/Dense>

using Eigen6Vector = Eigen::Matrix<float, 6, 1>;
using Eigen6by6Matrix = Eigen::Matrix<float, 6, 6>;
using Eigen3by3Matrix = Eigen::Matrix<float, 3, 3>;


__device__ Transformation exponential_rotation(Vector3& w, float theta) {
  /* See page 113 of Modern Robotics.
  * I + sin θ[ωˆ] + (1 − cos θ)[ωˆ]2
  */

  Transformation identity = Transformation();
  Transformation sin_theta_w = Transformation().bracket_w(w) * sin(theta);
  Transformation cos_theta_w_squared = Transformation().bracket_w_squared(w) * (1.0 - cos(theta));

  return identity + sin_theta_w + cos_theta_w_squared;
}

__device__ Vector3 rigid_body_exponential_translation(Vector3& w, Vector3& v, float theta) {
  /* See page 113 of Modern Robotics.
   * (Iθ + (1 − cos θ)[ω] + (θ − sin θ)[ω]2)v
   * (A + B + C) * (v)
   * (3 x 3 matrix) * (3 x 1 matrix) = (3 x 1 matrix)
   */

  Transformation identity_theta = Transformation() * theta;
  Transformation sin_theta_w = Transformation().bracket_w(w) * (1.0 - cos(theta));
  Transformation cos_theta_w_squared = Transformation().bracket_w_squared(w) * (theta - sin(theta));

  return (identity_theta + sin_theta_w + cos_theta_w_squared) * v;
}

__device__ Transformation multiply(Transformation* transformations, int n) {

  // Multiply each transformation together.
  Transformation result = Transformation();
  for (int i = 0; i < n; i++) {
    result = result * transformations[i];
  }

  return result;
}

__device__ Transformation rigid_body_exponential(Screw& screw, float theta) {

  // Create the transformations.
  Transformation result = exponential_rotation(screw.w, theta);
  Vector3 translatation = rigid_body_exponential_translation(screw.w, screw.v, theta);

  result.set_translation(translatation);
  return result;
}

__device__ Transformation forward_kinematics(Screw* screws, float* thetas, Transformation& home_position, int num_joints) {

  int num_transformations = num_joints + 1;
  Transformation* transformations = (Transformation*) malloc(sizeof(Transformation) * num_transformations);
  for (int i = 0; i < num_joints; i++) {
    screws[i].w.normalize();
    transformations[i] = rigid_body_exponential(screws[i], thetas[i]);
  }
  transformations[num_joints] = home_position;
  return multiply(transformations, num_transformations);
}

__device__ Eigen6by6Matrix jacobian(Screw* screws, float* thetas, Transformation& home_position, int num_joints) {

  Transformation* transformations = (Transformation*) malloc(sizeof(Transformation) * (num_joints + 1));
  Eigen6by6Matrix jacobian_matrix(num_joints, 6);
  for (int i = 0; i < num_joints; i++) {
    screws[i].w.normalize();
    transformations[i] = rigid_body_exponential(screws[i], thetas[i]);
  }
  transformations[num_joints] = home_position;

  Transformation result_original = multiply(transformations, num_joints + 1);
  Eigen6Vector pose_original = result_original.pose().to_eigen();

  // Move into parameterized struct.
  float delta = 0.0001;

  // Multiply each transformation together.
  for (int i = 0; i < num_joints; i++) {

    Transformation original = transformations[i];
    Transformation altered_transformation = rigid_body_exponential(screws[i], thetas[i] + delta);
    transformations[i] = altered_transformation;

    Transformation result = multiply(transformations, num_joints + 1);

    // Add a column of the jacobian matrix.
    Eigen6Vector column = result.pose().to_eigen() - pose_original;
    jacobian_matrix.col(i) = column / delta;

    transformations[i] = original;
  }


  //free(transformations);
  //return &result;
  return jacobian_matrix;
}

__device__ Eigen3by3Matrix inverse(Eigen3by3Matrix& matrix) {

  float a, b, c, d, e, f, g, h, i;
  float* values = matrix.data(); // Column major.
  a = values[0];
  d = values[1];
  g = values[2];

  b = values[3];
  e = values[4];
  h = values[5];

  c = values[6];
  f = values[7];
  i = values[8];

  float epsilon = 0.00001;
  float divisor = (a * (e * i - f * h)) + (-b * (d * i - f * g)) + (c * (d * h - e * g));
  if (fabsf(divisor) < epsilon) {
    divisor += epsilon;
  }

  Eigen3by3Matrix result;
  result(0, 0) = e * i - f * h;
  result(0, 1) = c * h - b * i;
  result(0, 2) = b * f - c * e;

  result(1, 0) = f * g - d * i;
  result(1, 1) = a * i - c * g;
  result(1, 2) = c * d - a * f;

  result(2, 0) = d * h - e * g;
  result(2, 1) = b * g - a * h;
  result(2, 2) = a * e - b * d;

  result = result / divisor;

  return result;
}

__device__ Eigen6by6Matrix inverse(Eigen6by6Matrix& matrix) {
  // Eigen only supports GPU inverse up to matrices of size 4x4.
  // See https://en.wikipedia.org/wiki/Block_matrix
  using namespace Eigen;

  Eigen3by3Matrix A = matrix.topLeftCorner(3, 3);
  Eigen3by3Matrix B = matrix.topRightCorner(3, 3);
  Eigen3by3Matrix C = matrix.bottomLeftCorner(3, 3);
  Eigen3by3Matrix D = matrix.bottomRightCorner(3, 3);

  Eigen3by3Matrix A_i = inverse(A);
  Eigen3by3Matrix B_i = inverse(B);
  Eigen3by3Matrix C_i = inverse(C);
  Eigen3by3Matrix D_i = inverse(D);

  Eigen3by3Matrix A_i_B = A_i * B;
  Eigen3by3Matrix C_A_i_B = C * A_i_B;
  Eigen3by3Matrix C_A_i = C * A_i;
  Eigen3by3Matrix aaa = D - C_A_i_B;
  Eigen3by3Matrix D_minus_C_A_i_B_inverse = inverse(aaa);

  Eigen3by3Matrix A_result = A_i + (A_i_B * D_minus_C_A_i_B_inverse) * C_A_i;

  Eigen3by3Matrix B_result = (-1 * A_i_B) * D_minus_C_A_i_B_inverse;

  Eigen3by3Matrix C_result = -1 * D_minus_C_A_i_B_inverse * C_A_i;

  Eigen3by3Matrix D_result = D_minus_C_A_i_B_inverse;

  Eigen6by6Matrix result;
  result.topLeftCorner<3, 3>() = A_result;
  result.topRightCorner<3, 3>() = B_result;
  result.bottomLeftCorner<3, 3>() = C_result;
  result.bottomRightCorner<3, 3>() = D_result;

  return result;
}

Eigen6by6Matrix inverse_cpu(Eigen6by6Matrix& matrix) {
  // Eigen only supports GPU inverse up to matrices of size 4x4.
  // See https://en.wikipedia.org/wiki/Block_matrix
  using namespace Eigen;

  Eigen3by3Matrix A = matrix.topLeftCorner(3, 3);
  Eigen3by3Matrix B = matrix.topRightCorner(3, 3);
  Eigen3by3Matrix C = matrix.bottomLeftCorner(3, 3);
  Eigen3by3Matrix D = matrix.bottomRightCorner(3, 3);

  Eigen3by3Matrix A_i = A.inverse();
  Eigen3by3Matrix B_i = B.inverse();
  Eigen3by3Matrix C_i = C.inverse();
  Eigen3by3Matrix D_i = D.inverse();

  Eigen3by3Matrix A_i_B = A_i * B;
  Eigen3by3Matrix C_A_i_B = C * A_i_B;
  Eigen3by3Matrix C_A_i = C * A_i;
  Eigen3by3Matrix D_minus_C_A_i_B_inverse = (D - C_A_i_B).inverse();

  Eigen3by3Matrix A_result = A_i + (A_i_B * D_minus_C_A_i_B_inverse) * C_A_i;

  Eigen3by3Matrix B_result = (-1 * A_i_B) * D_minus_C_A_i_B_inverse;

  Eigen3by3Matrix C_result = -1 * D_minus_C_A_i_B_inverse * C_A_i;

  Eigen3by3Matrix D_result = D_minus_C_A_i_B_inverse;

  Eigen6by6Matrix result;
  result.topLeftCorner<3, 3>() = A_result;
  result.topRightCorner<3, 3>() = B_result;
  result.bottomLeftCorner<3, 3>() = C_result;
  result.bottomRightCorner<3, 3>() = D_result;

  return result;
}

__device__ float absolute_error(Eigen6Vector& left, Eigen6Vector& right) {

  return (left - right).norm();
}

__device__ Eigen6Vector inverse_kinematics(Screw* screws, float* thetas, Transformation& home_position, int num_joints) {

  // Move into configurable struct.
  int max_num_iterations = 200;
  int max_allowed_error = 0.000001;
  float epsilon = 0.25;

  Eigen6Vector desired_pose;
  desired_pose << 0.0957158, 0.109065 + 0.12, 0.987924, 0., 0., 3.14/2.0;
  Eigen6Vector current_pose = forward_kinematics(screws, thetas, home_position, num_joints).pose().to_eigen();

  int i = 0;
  while (i < max_num_iterations && absolute_error(current_pose, desired_pose) > max_allowed_error) {

    // Calculate how to update thetas.
    current_pose = forward_kinematics(screws, thetas, home_position, num_joints).pose().to_eigen();
    Eigen6by6Matrix jacobian_matrix = jacobian(screws, thetas, home_position, num_joints);
    Eigen6by6Matrix jacobian_matrix_inverse = inverse(jacobian_matrix);
    Eigen::VectorXf theta_deltas = jacobian_matrix_inverse * (desired_pose - current_pose);

    // Apply the theta updates.
    for (int j = 0; j < num_joints; j++) {
      float update = theta_deltas[j] * epsilon / (theta_deltas.norm() + 0.000001);
      thetas[j] += update;
    }

    i = i + 1;
  }


  Eigen6Vector result_thetas;
  result_thetas << thetas[0], thetas[1], thetas[2], thetas[3], thetas[4], thetas[5];
  //return result_thetas;
  return current_pose;
}

__device__ Transformation test_inverse(Transformation* result) {

  Transformation identity = Transformation();

  Vector3 t;
  t.x = 1;
  t.y = 2;
  t.z = 3;
  identity.set_translation(t);

  float* values = (float*) malloc(sizeof(float) * 6 * 6);
  for (int i = 0; i < 6; i++) {
    for (int j = 0; j < 6; j++) {
      values[(i * 6) + j] = 0.0;
      if (i == j) {
        values[(i * 6) + j] = 1.0;
      }
    }
  }

  //values[0] = 10.0;
  //values[6] = 4.0;
  //values[12] = 14.0;
  //values[18] = 3.0;
  //values[24] = 0.5;
  //values[30] = 22.0;

  Eigen::Map<Eigen::Matrix<float, 6, 6, Eigen::RowMajor>> eigenMatrix(values);
  Eigen6by6Matrix a = eigenMatrix;
  Eigen::Matrix<float, 6, 6> invertedMatrix = inverse(a);

  result[0] = Transformation(invertedMatrix.data());
}

__global__ void test_inverse_kernel(Transformation* result) {

  test_inverse(result);
}

__global__ void forward_kinematics_kernel(Screw* screw_axes, float* thetas, float* home_position_values, int num_joints, int num_problems, Transformation* result_transformations) {

  int inputs_index = (blockIdx.x * blockDim.x) + threadIdx.x;
  int result_index = inputs_index;
  int offset = blockDim.x * gridDim.x;

  Transformation home_position = Transformation(home_position_values);

  while (inputs_index < num_problems) {
    // Screw* screws, float* thetas, int num_joints, float* home_position_values
    result_transformations[result_index] = forward_kinematics(screw_axes, thetas + (inputs_index * num_joints), home_position, num_joints);

    inputs_index += offset;
    result_index += offset;
  }
}

__global__ void inverse_kinematics_kernel(Screw* screw_axes, float* thetas, float* home_position_values, int num_joints, int num_problems, Eigen6Vector* results) {

  int inputs_index = 0;
  int result_index = 0;

  Transformation home_position = Transformation(home_position_values);
  results[result_index] = inverse_kinematics(screw_axes, thetas + (num_joints * inputs_index), home_position, num_joints);
}


void create_joint_angles(float a, float b, float c, float d, float e, float f, float* output) {
  output[0] = a;
  output[1] = b;
  output[2] = c;
  output[3] = d;
  output[4] = e;
  output[5] = f;
}

int main() {

  // Robot info.
  int num_joints = 6;
  int num_configurations = 1; //10550;

  Transformation* result_transformation_host = (Transformation*) malloc(sizeof(Transformation) * num_configurations);
  Transformation* result_transformation_device;
  cudaMalloc((void**)&result_transformation_device, sizeof(Transformation) * num_configurations);

  Eigen6Vector* ik_results_host = (Eigen6Vector*) malloc(sizeof(Eigen6Vector) * num_configurations);
  Eigen6Vector* ik_results_device;
  cudaMalloc((void**)&ik_results_device, sizeof(Eigen6Vector) * num_configurations);

  Screw* screw_axes_host = (Screw*) malloc(sizeof(Screw) * num_joints);
  Screw* screw_axes_device;
  cudaMalloc((void**)&screw_axes_device, sizeof(Screw) * num_joints);

  float* M_host = (float*) malloc(sizeof(float) * Transformation::num_entries_in_matrix);
  float* M_device;
  cudaMalloc((void**)&M_device, sizeof(float) * Transformation::num_entries_in_matrix);

  float* thetas_host = (float*) malloc(sizeof(float) * num_joints * num_configurations);
  float* thetas_device;
  cudaMalloc((void**)&thetas_device, sizeof(float) * num_joints * num_configurations);

  // UR5 Screw axes.
  // See page 148 of Modern Robotics.
  float H1 = 0.089;
  float H2 = 0.095;
  float L1 = 0.425;
  float L2 = 0.392;
  float W1 = 0.109;
  float W2 = 0.082;

  // Define the screw axes.
  screw_axes_host[0] = Screw(Vector3(0., 0., 1.), Vector3(0., 0., 0.));
  screw_axes_host[1] = Screw(Vector3(0., 1., 0.), Vector3(-H1, 0., 0.));
  screw_axes_host[2] = Screw(Vector3(0., 1., 0.), Vector3(-H1, 0., L1));
  screw_axes_host[3] = Screw(Vector3(0., 1., 0.), Vector3(-H1, 0., L1 + L2));
  screw_axes_host[4] = Screw(Vector3(0., 0., -1.), Vector3(-W1, L1 + L2, 0.));
  screw_axes_host[5] = Screw(Vector3(0., 1., 0.), Vector3(H2 - H1, 0., L1 + L2));

  // Define the home pose.
  M_host[0] = -1.;
  M_host[1] = 0.;
  M_host[2] = 0.;
  M_host[3] = L1 + L2;

  M_host[4] = 0.;
  M_host[5] = 0.;
  M_host[6] = 1.;
  M_host[7] = W1 + W2;

  M_host[8] = 0.;
  M_host[9] = 1.;
  M_host[10] = 0.;
  M_host[11] = H1 - H2;

  M_host[12] = 0.;
  M_host[13] = 0.;
  M_host[14] = 0.;
  M_host[15] = 1.;

  for (int i = 0; i < num_configurations; i++) {
    create_joint_angles(0., -3.14/2.0, 0., 0., 3.14/2.0, 0., thetas_host + (num_joints * i));
  }

  // Copy the screw axes onto the device.
  cudaMemcpy(screw_axes_device, screw_axes_host, sizeof(Screw) * num_joints, cudaMemcpyHostToDevice);

  // Copy M onto the device.
  cudaMemcpy(M_device, M_host, sizeof(float) * Transformation::num_entries_in_matrix, cudaMemcpyHostToDevice);

  // Copy the joint angles onto the device.
  cudaMemcpy(thetas_device, thetas_host, sizeof(float) * num_joints * num_configurations, cudaMemcpyHostToDevice);

  // Screw* screw_axes, float* thetas_device, float* home_position_values, int num_joints, Transformation* result_transformations
  //forward_kinematics_kernel<<<1, 1>>>(screw_axes_device, thetas_device, M_device, num_joints, num_configurations, result_transformation_device);


  //test_inverse_kernel<<<1, 1>>>(result_transformation_device);

  inverse_kinematics_kernel<<<1, 1>>>(screw_axes_device, thetas_device, M_device, num_joints, num_configurations, ik_results_device);

  int a = cudaMemcpy(result_transformation_host, result_transformation_device, sizeof(Transformation) * num_configurations, cudaMemcpyDeviceToHost);
  std::cout << a << std::endl;

  cudaMemcpy(ik_results_host, ik_results_device, sizeof(Eigen6Vector) * num_configurations, cudaMemcpyDeviceToHost);

  // Display one of the Transformations.
  //result_transformation_host[num_configurations - 1].print();
  std::cout << *ik_results_host << std::endl;

  float* values = (float*) malloc(sizeof(float) * 6 * 6);
  for (int i = 0; i < 6; i++) {
    for (int j = 0; j < 6; j++) {
      values[(i * 6) + j] = 0.0;
      if (i == j) {
        values[(i * 6) + j] = 1.0;
      }
    }
  }

  Eigen::Map<Eigen::Matrix<float, 6, 6, Eigen::RowMajor>> eigenMatrix(values);
  Eigen6by6Matrix aaa = eigenMatrix;
  aaa(0, 3) = 5.0;
  aaa(3, 4) = 30.0;

  std::cout << aaa << std::endl << std::endl;

  Eigen::Matrix<float, 6, 6> invertedMatrix = inverse_cpu(aaa);

  std::cout << invertedMatrix << std::endl;

  // Free the device memory.
  cudaFree(screw_axes_device);
  cudaFree(thetas_device);
  cudaFree(result_transformation_device);
  cudaFree(M_device);

  // Free the host memory.
  free(screw_axes_host);
  free(result_transformation_host);
  free(M_host);

  return 0;
}

Overwriting main.cu


In [ ]:
%%script bash
#apt install libeigen3-dev
#ln -sf /usr/include/eigen3/Eigen /usr/include/Eigen
#nvcc screw.cu vectors.cu transformation.cu -dc

# nvcc main.cu screw.cu vectors.cu transformation.cu -rdc=true -o forward_kinematics
#nvcc screw.cu vectors.cu transformation.cu main.cu -rdc=true -o forward_kinematics
nvcc main.cu transformation.cu vectors.cu screw.cu -rdc=true -o forward_kinematics

#nvcc main.cu -o forward_kinematics -dynamic NVCC_FLAGS=-lcublas_device -L/usr/local/cuda/lib64 -lcublas_device
#nvcc main.cu -lcublas_static -lculibos -o forward_kinematics

In [ ]:
%%script bash
./forward_kinematics